In [ ]:
include("init.jl")
import HTTP
import JSON
import Distributed
import Dates
import TimeSeries
using Caching

In [ ]:
const API_KEY = "bfb480074e9b9499c7627f81ca467b2c"
const LIBIO_DATE_FORMAT = Dates.DateFormat("y-m-d\\TH:M:S.s\\Z")

In [ ]:
# Func to request from API and rate limit

function api_get(url, key=API_KEY)
    return _api_get(url, key) |> JSON.parse
end

@cache function _api_get(url::String, key::String=API_KEY)
    while true
        try
            response = HTTP.get(url * "&api_key=$key")
            return response.body |> String
        catch (e)
            if e isa HTTP.ExceptionRequest.StatusError
                if e.status == 429
                    println(429)
                    sleep(1)
                    # return api_get(url, key)
                else
                    throw(e)
                end
            else
                throw(e)
            end
        end
    end
end "libraries-io-api-get.bin"

function dateconv(libiodatestr)
    Dates.DateTime(libiodatestr,LIBIO_DATE_FORMAT)
end

function deps_get(pkgname,pkgmanager)
    sanitary_pkgname = HTTP.URIs.escapeuri(pkgname)
    url = (version) -> "https://libraries.io/api/$(pkgmanager)/$(sanitary_pkgname)/" * HTTP.URIs.escapeuri(version) * "/dependencies?"
    pkgdeets =  "latest" |> url |> api_get |> deepcopy
    for v in pkgdeets["versions"] # yeah, we do sort of waste a call here, but who cares?
        vdeets = v["number"] |> url |> api_get |> deepcopy
        v["dependencies"] = [v["name"] for v in vdeets["dependencies"]]
        v["published_at_datetime"] = v["published_at"] |> dateconv
    end
    return pkgdeets
end
# So - we need to decide where/how to store this information. I'm tempted to just chuck it in the main JSON/dict, under pkg[version_number].
#[c["number"] for c in core["versions"]]
#(v -> findfirst(x->x["number"]==v, core["versions"]))("5.1.1")

In [ ]:
# Func to get all versions

function paginated_get(url, key=API_KEY)
    # Api get page 1, sleep .5, get page 2, continue until 404.
    results = []
    page = 1
    while true
        try
            r =  api_get(url * "&page=$page&per_page=100&api_key=$key") |> deepcopy
            length(r) == 0 && return results
            push!(results, r)
            print(page, " ")
            page += 1
        catch e
            if e isa HTTP.ExceptionRequest.StatusError
                if e.status == 404
                    return results
                elseif e.status == 500
                    # Sometimes we'll get a 500 error. If we do we should probably just try again.
                else
                    # THIS IS BAD don't do this
                    print(e)
                    page += 1
                end
            else throw(e)
            end
        end
        sleep(.5)
    end
end

In [ ]:
#elm_versions_task = Distributed.@spawn paginated_get("https://libraries.io/api/search?&platforms=elm")

In [ ]:
@syncache! _api_get

In [ ]:
api_get("https://libraries.io/api/search?&platforms=elm")

In [ ]:
#fetch(elm_versions_task)

In [ ]:
@persist! api_get

In [ ]:
import TranscodingStreams, CodecZlib

In [ ]:
# Save package data 

#open("elm_pkg_cache.json.gz","w") do f
#    write(f,TranscodingStreams.transcode(
#            CodecZlib.GzipCompressor,Vector{UInt8}(JSON.json(fetch(elm_versions_task),4))
#        )
#    )
#end;

In [ ]:
# Load package data from file to stop us hammering NPM
elm_cache = open("elm_pkg_cache.json.gz","r") do f
    JSON.parse(String(TranscodingStreams.transcode(
            CodecZlib.GzipDecompressor,read(f)
    )))
end;

In [ ]:
core = deps_get("elm-lang/core","elm")

In [ ]:
api_get

In [ ]:
td = core["versions"][1]
td.slots

In [ ]:
[c["number"] for c in core["versions"]]
#(v -> findfirst(x->x["number"]==v, core["versions"]))("5.1.1")

In [ ]:
(v -> findfirst(x->x["number"]==v, core["versions"]))("5.1.1")

In [ ]:
a = Dict()
a["ok"] = "test"

In [ ]:
elmcss = deps_get("rtfeldman/elm-css","elm")

In [ ]:
#for v in elmcss["versions"]
#    println(v["published_at"], " ", v["number"], " ", v["dependencies"])
#end
sort!(elmcss["versions"],by=x->x["published_at"])
elmts = TimeSeries.TimeArray(
    [v["published_at"] for v in elmcss["versions"]],
    [v["number"] for v in elmcss["versions"]],
)

In [ ]:
TimeSeries.to(
    TimeSeries.collapse(elmts,TimeSeries.month,TimeSeries.last),
    Dates.DateTime("2016-05-11")
)[end]

In [ ]:
TimeSeries.collapse(elmts,TimeSeries.month,TimeSeries.last),